<a href="https://colab.research.google.com/github/meti-94/CodingQuestions/blob/main/Predictive%20Analysis/Features%20Combination/Text-Image%20Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle -q
!pip install -q -U "tensorflow-text==2.8.*"
!pip install -q tf-models-official==2.7.0
!pip install tensorflow-addons -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 498.0/498.0 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.3/462.3 KB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 69.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [3]:
!mkdir ~/.kaggle
!cp /content/gdrive/MyDrive/kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d williamscott701/memotion-dataset-7k

 99% 689M/695M [00:04<00:00, 175MB/s]
100% 695M/695M [00:04<00:00, 162MB/s]


In [5]:
!unzip -qq /content/memotion-dataset-7k.zip

In [2]:
import re
import string
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Activation, Dropout
from tensorflow.keras.layers import Conv1D, Embedding, GlobalAveragePooling1D 
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing import image
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

import tensorflow_addons as tfa

import tensorflow_hub as hub
import tensorflow_text as text

from PIL import ImageFile, ImageOps
ImageFile.LOAD_TRUNCATED_IMAGES = True

/usr/local/lib/python3.8/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.4 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [3]:
df = pd.read_csv('/content/memotion_dataset_7k/labels.csv')
df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
df = df.drop(columns = ['text_ocr'])
df.head()

,image_name,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment
0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,hilarious,general,not_offensive,not_motivational,very_positive
1,image_2.jpeg,The best of #10 YearChallenge! Completed in le...,not_funny,general,not_offensive,motivational,very_positive
2,image_3.JPG,Sam Thorne @Strippin ( Follow Follow Saw every...,very_funny,not_sarcastic,not_offensive,not_motivational,positive
3,image_4.png,10 Year Challenge - Sweet Dee Edition,very_funny,twisted_meaning,very_offensive,motivational,positive
4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,hilarious,very_twisted,very_offensive,not_motivational,neutral


In [4]:
df[df.isnull().any(axis=1)]

,image_name,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment
119,image_120.jpg,NaN,not_funny,general,not_offensive,not_motivational,positive
4799,image_4800.jpg,NaN,very_funny,general,slight,motivational,neutral
6781,image_6782.jpg,NaN,very_funny,twisted_meaning,not_offensive,not_motivational,positive
6784,image_6785.jpg,NaN,hilarious,general,not_offensive,not_motivational,positive
6786,image_6787.jpg,NaN,not_funny,not_sarcastic,very_offensive,motivational,positive


In [5]:
full_df = df.copy()
full_df.isnull().any()

image_name           False
text_corrected        True
humour               False
sarcasm              False
offensive            False
motivational         False
overall_sentiment    False
dtype: bool

In [6]:
clean_df = df.copy()
clean_df.dropna(inplace=True)
clean_df.isnull().any()

image_name           False
text_corrected       False
humour               False
sarcasm              False
offensive            False
motivational         False
overall_sentiment    False
dtype: bool

In [7]:
df = df.replace({'humour': {'not_funny': 0, 'funny': 1, 'very_funny': 2, 'hilarious':3},
            'sarcasm': {'not_sarcastic': 0, 'general': 1, 'twisted_meaning': 2, 'very_twisted': 3},
            'offensive': {'not_offensive': 0, 'slight': 1, 'very_offensive': 2, 'hateful_offensive': 3},
            'motivational': {'not_motivational': 0, 'motivational': 1},
            'overall_sentiment': {'very_negative': 0, 'negative': 1, 'neutral': 2, 'positive': 3, 'very_positive': 4}})

In [8]:
width = 100
height = 100
X_full = []
X_full_path = []
for i in tqdm(range(full_df.shape[0])):
    path = '/content/memotion_dataset_7k/images/'+full_df['image_name'][i]
    img = image.load_img(path,target_size=(width,height,3))
    img = image.img_to_array(img)
    img = img/255.0
    X_full.append(img)
    X_full_path.append(path)

 93%|█████████▎| 6501/6992 [00:51<00:03, 156.80it/s]/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))
 95%|█████████▌| 6670/6992 [00:52<00:02, 122.21it/s]/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
100%|██████████| 6992/6992 [00:54<00:00, 127.51it/s]


In [9]:
width = 100
height = 100
X_cleaned = []
X_cleaned_path = []
for i in tqdm(range(clean_df.shape[0])):
    if i in [119, 4799, 6781, 6784, 6786]:
        pass
    else:
        path = '/content/memotion_dataset_7k/images/'+clean_df['image_name'][i]
        img = image.load_img(path,target_size=(width,height,3))
        img = image.img_to_array(img)
        img = img/255.0
        X_cleaned.append(img)
        X_cleaned_path.append(path)

100%|██████████| 6987/6987 [00:47<00:00, 146.33it/s]


In [10]:
deleted_paths = [paths for paths in X_full_path + X_cleaned_path if paths not in X_full_path or paths not in X_cleaned_path]
deleted_paths

['/content/memotion_dataset_7k/images/image_120.jpg',
 '/content/memotion_dataset_7k/images/image_4800.jpg',
 '/content/memotion_dataset_7k/images/image_6782.jpg',
 '/content/memotion_dataset_7k/images/image_6785.jpg',
 '/content/memotion_dataset_7k/images/image_6787.jpg',
 '/content/memotion_dataset_7k/images/image_6988.jpg',
 '/content/memotion_dataset_7k/images/image_6989.jpg',
 '/content/memotion_dataset_7k/images/image_6990.png',
 '/content/memotion_dataset_7k/images/image_6991.jpg',
 '/content/memotion_dataset_7k/images/image_6992.jpg']

In [11]:
delete_var = [full_df, clean_df, X_cleaned, X_full, X_cleaned_path, X_full_path]
for i in delete_var:
    del i

In [12]:
len(clean_df)

6987

In [13]:
width = 100
height = 100
X = []
for i in tqdm(range(clean_df.shape[0])):
    if i in [119, 4799, 6781, 6784, 6786]:
        pass
    else:
        path = '/content/memotion_dataset_7k/images/'+clean_df['image_name'][i]
        img = image.load_img(path,target_size=(width,height,3))
        img = image.img_to_array(img)
        img = img/255.0
        X.append(img)

100%|██████████| 6987/6987 [00:47<00:00, 146.62it/s]


In [14]:
X = np.array(X)

In [15]:
X.shape

(6982, 100, 100, 3)

In [16]:
rows_to_drop = ['image_120.jpg',
              'image_4800.jpg',
              'image_6782.jpg',
              'image_6785.jpg',
              'image_6787.jpg',
              'image_6988.jpg',
              'image_6989.jpg',
              'image_6990.png',
              'image_6991.jpg',
              'image_6992.jpg']

In [17]:
cleaner_df = df
cleaner_df.head()

,image_name,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment
0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,3,1,0,0,4
1,image_2.jpeg,The best of #10 YearChallenge! Completed in le...,0,1,0,1,4
2,image_3.JPG,Sam Thorne @Strippin ( Follow Follow Saw every...,2,0,0,0,3
3,image_4.png,10 Year Challenge - Sweet Dee Edition,2,2,2,1,3
4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,3,3,2,0,2


In [18]:
len(cleaner_df)

6992

In [19]:
for images in rows_to_drop:
    cleaner_df.drop(cleaner_df[cleaner_df['image_name'] == images].index, inplace=True)

In [20]:
cleaner_df.shape

(6982, 7)

In [21]:
np.save('image_array', X)

In [22]:
Y = cleaner_df.iloc[:,2:]
Y.shape

(6982, 5)

In [23]:
X.shape, Y.shape

((6982, 100, 100, 3), (6982, 5))

In [24]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

preprocess_input = tf.keras.applications.resnet_v2.preprocess_input

rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)

In [25]:
base_model_1 = tf.keras.applications.ResNet50(input_shape=X[0].shape,
                                               include_top=False,
                                               weights='imagenet')
base_model_2 = tf.keras.applications.VGG16(input_shape=X[0].shape,
                                               include_top=False,
                                               weights='imagenet')

In [26]:
base_model_1.trainable = False
base_model_2.trainable = False

In [27]:
global_average_layer = GlobalAveragePooling2D()

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state=42)

In [29]:
del X

In [30]:
image_input = tf.keras.Input(shape=(100, 100, 3), name = 'image_input')
iamge_layers = data_augmentation(image_input)
iamge_layers = preprocess_input(iamge_layers)
layer_bm_1 = base_model_1(image_input, training=False)
layer_bm_1 = Conv2D(2048, kernel_size=2,padding='valid')(layer_bm_1)
layer_bm_1 = Dense(512)(layer_bm_1)
layer_bm_2 = base_model_2(image_input, training=False)
layer_bm_2 = Dense(512)(layer_bm_2)
layers = tf.keras.layers.concatenate([layer_bm_1, layer_bm_2])
iamge_layers = global_average_layer(layer_bm_1)
image_layers = Dropout(0.2, name = 'dropout_layer')(iamge_layers)

In [31]:
cleaner_df.shape

(6982, 7)

In [32]:
def standardization(data):
    data = data.apply(lambda x: x.lower())
    data = data.apply(lambda x: re.sub(r'\d+', '', x))
    data = data.apply(lambda x: re.sub(r'.com', '', x, flags=re.MULTILINE))
    data = data.apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
    return data

df['text_corrected'] = standardization(cleaner_df.text_corrected)

In [33]:
X_text_train, X_text_test, y_text_train, y_text_test = train_test_split(cleaner_df.text_corrected, Y, test_size = 0.2, random_state=42)

In [34]:
#@title Choose a BERT model to fine-tune

bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'  #@param ["bert_en_uncased_L-12_H-768_A-12", "bert_en_cased_L-12_H-768_A-12", "bert_multi_cased_L-12_H-768_A-12", "small_bert/bert_en_uncased_L-2_H-128_A-2", "small_bert/bert_en_uncased_L-2_H-256_A-4", "small_bert/bert_en_uncased_L-2_H-512_A-8", "small_bert/bert_en_uncased_L-2_H-768_A-12", "small_bert/bert_en_uncased_L-4_H-128_A-2", "small_bert/bert_en_uncased_L-4_H-256_A-4", "small_bert/bert_en_uncased_L-4_H-512_A-8", "small_bert/bert_en_uncased_L-4_H-768_A-12", "small_bert/bert_en_uncased_L-6_H-128_A-2", "small_bert/bert_en_uncased_L-6_H-256_A-4", "small_bert/bert_en_uncased_L-6_H-512_A-8", "small_bert/bert_en_uncased_L-6_H-768_A-12", "small_bert/bert_en_uncased_L-8_H-128_A-2", "small_bert/bert_en_uncased_L-8_H-256_A-4", "small_bert/bert_en_uncased_L-8_H-512_A-8", "small_bert/bert_en_uncased_L-8_H-768_A-12", "small_bert/bert_en_uncased_L-10_H-128_A-2", "small_bert/bert_en_uncased_L-10_H-256_A-4", "small_bert/bert_en_uncased_L-10_H-512_A-8", "small_bert/bert_en_uncased_L-10_H-768_A-12", "small_bert/bert_en_uncased_L-12_H-128_A-2", "small_bert/bert_en_uncased_L-12_H-256_A-4", "small_bert/bert_en_uncased_L-12_H-512_A-8", "small_bert/bert_en_uncased_L-12_H-768_A-12", "albert_en_base", "electra_small", "electra_base", "experts_pubmed", "experts_wiki_books", "talking-heads_base"]

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [35]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)
bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [36]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
encoder_inputs = preprocessing_layer(text_input)
encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
outputs = encoder(encoder_inputs)
text_layers = outputs['pooled_output']

text_layers = tf.keras.layers.Dense(2048, activation="relu")(text_layers)

In [37]:
concatenate = tf.keras.layers.concatenate([image_layers, text_layers], axis=1)

In [38]:
overall_layer = tf.keras.layers.Dense(2048, activation='softmax')(concatenate)

In [39]:
prediction_layer_1 = tf.keras.layers.Dense(4, activation='softmax', name = 'sarcasm')
prediction_layer_2 = tf.keras.layers.Dense(4, activation='softmax', name = 'humuor')
prediction_layer_3 = tf.keras.layers.Dense(4, activation='softmax', name = 'offensive')
prediction_layer_4 = tf.keras.layers.Dense(1, activation='sigmoid', name = 'motivational')
prediction_layer_5 = tf.keras.layers.Dense(5, activation='softmax', name = 'overall')

In [40]:
output_1 = prediction_layer_1(overall_layer)
output_2 = prediction_layer_2(overall_layer)
output_3 = prediction_layer_3(overall_layer)
output_4 = prediction_layer_4(overall_layer)
output_5 = prediction_layer_5(overall_layer)
model = tf.keras.Model(inputs = [image_input, text_input] , outputs = [output_1, output_2, output_3, output_4, output_5])

In [41]:

losses = {
      "humuor": tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
      "sarcasm": tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
      "offensive": tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
      "motivational": tf.keras.losses.BinaryCrossentropy(from_logits=False),
      "overall": tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
}
lossWeights = {
      "humuor": 1.0, 
      "sarcasm": 1.0, 
      "offensive": 1.0, 
      "motivational": 1.0,
      "overall": 1.0
}
metric = {
    "humuor": ['acc',tfa.metrics.F1Score(num_classes=4, average="micro", threshold = 0.9)],
    "sarcasm": ['acc',tfa.metrics.F1Score(num_classes=4, average="micro", threshold = 0.9)],
    "offensive": ['acc',tfa.metrics.F1Score(num_classes=4, average="micro", threshold = 0.9)],
    "motivational": ['acc',tfa.metrics.F1Score(num_classes=1, average="micro", threshold = 0.9)],
    "overall": ['acc',tfa.metrics.F1Score(num_classes=5, average="micro", threshold = 0.9)]
}


In [42]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='min')
mcp_save = ModelCheckpoint('model-{epoch:03d}-{overall_accuracy:03f}-{val_overall_accuracy:03f}', save_format='tf', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=27, verbose=1, epsilon=1e-4, mode='min')

In [44]:


learning_rates = [0.001] # [0.1, 0.01, 0.001, 0.0001, 0.00001]
for i in learning_rates:
    model.compile(optimizer=tf.keras.optimizers.RMSprop(i),
              loss = losses,
              loss_weights= lossWeights,
              metrics=['accuracy'])
    
    history = model.fit(x = {"image_input": X_train, "text": X_text_train},
                    y = {"sarcasm": y_train.sarcasm, 
                         "humuor": y_train.humour, 
                         "offensive": y_train.offensive, 
                         "motivational": y_train.motivational, 
                         "overall": y_train.overall_sentiment},
                    batch_size=64,
                    epochs=50,
                    verbose=1,
                    validation_split=0.2,
                    callbacks=[earlyStopping, mcp_save,reduce_lr_loss]
                   )
     
     


Epoch 1/50


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


70/70 [==============================] - ETA: 0s - loss: 6.2660 - sarcasm_loss: 1.3359 - humuor_loss: 1.3564 - offensive_loss: 1.3353 - motivational_loss: 0.6885 - overall_loss: 1.5499 - sarcasm_accuracy: 0.4944 - humuor_accuracy: 0.3581 - offensive_accuracy: 0.3693 - motivational_accuracy: 0.5748 - overall_accuracy: 0.4425

70/70 [==============================] - 84s 1s/step - loss: 6.2660 - sarcasm_loss: 1.3359 - humuor_loss: 1.3564 - offensive_loss: 1.3353 - motivational_loss: 0.6885 - overall_loss: 1.5499 - sarcasm_accuracy: 0.4944 - humuor_accuracy: 0.3581 - offensive_accuracy: 0.3693 - motivational_accuracy: 0.5748 - overall_accuracy: 0.4425 - val_loss: 6.1428 - val_sarcasm_loss: 1.3034 - val_humuor_loss: 1.3431 - val_offensive_loss: 1.3120 - val_motivational_loss: 0.6812 - val_overall_loss: 1.5031 - val_sarcasm_accuracy: 0.5157 - val_humuor_accuracy: 0.3509 - val_offensive_accuracy: 0.3787 - val_motivational_accuracy: 0.6321 - val_overall_accuracy: 0.4628 - lr: 0.0010
Epoch 2/50
70/70 [==============================] - ETA: 0s - loss: 6.0563 - sarcasm_loss: 1.2887 - humuor_loss: 1.3293 - offensive_loss: 1.2901 - motivational_loss: 0.6730 - overall_loss: 1.4752 - sarcasm_accuracy: 0.4944 - humuor_accuracy: 0.3581 - offensive_accuracy: 0.3693 - motivational_accuracy: 0.6486 - overall_accuracy: 0.4465

70/70 [==============================] - 71s 1s/step - loss: 6.0563 - sarcasm_loss: 1.2887 - humuor_loss: 1.3293 - offensive_loss: 1.2901 - motivational_loss: 0.6730 - overall_loss: 1.4752 - sarcasm_accuracy: 0.4944 - humuor_accuracy: 0.3581 - offensive_accuracy: 0.3693 - motivational_accuracy: 0.6486 - overall_accuracy: 0.4465 - val_loss: 5.9604 - val_sarcasm_loss: 1.2583 - val_humuor_loss: 1.3231 - val_offensive_loss: 1.2753 - val_motivational_loss: 0.6706 - val_overall_loss: 1.4332 - val_sarcasm_accuracy: 0.5157 - val_humuor_accuracy: 0.3509 - val_offensive_accuracy: 0.3787 - val_motivational_accuracy: 0.6321 - val_overall_accuracy: 0.4628 - lr: 0.0010
Epoch 3/50
70/70 [==============================] - ETA: 0s - loss: 5.9023 - sarcasm_loss: 1.2527 - humuor_loss: 1.3120 - offensive_loss: 1.2566 - motivational_loss: 0.6624 - overall_loss: 1.4187 - sarcasm_accuracy: 0.4944 - humuor_accuracy: 0.3581 - offensive_accuracy: 0.3738 - motivational_accuracy: 0.6486 - overall_accuracy: 0.4465

...

70/70 [==============================] - 72s 1s/step - loss: 5.5738 - sarcasm_loss: 1.1834 - humuor_loss: 1.2886 - offensive_loss: 1.1709 - motivational_loss: 0.6483 - overall_loss: 1.2825 - sarcasm_accuracy: 0.4944 - humuor_accuracy: 0.3581 - offensive_accuracy: 0.3926 - motivational_accuracy: 0.6486 - overall_accuracy: 0.4465 - val_loss: 5.5389 - val_sarcasm_loss: 1.1533 - val_humuor_loss: 1.2962 - val_offensive_loss: 1.1771 - val_motivational_loss: 0.6585 - val_overall_loss: 1.2539 - val_sarcasm_accuracy: 0.5157 - val_humuor_accuracy: 0.3509 - val_offensive_accuracy: 0.3724 - val_motivational_accuracy: 0.6321 - val_overall_accuracy: 0.4628 - lr: 0.0010
Epoch 19/50
70/70 [==============================] - ETA: 0s - loss: 5.5736 - sarcasm_loss: 1.1835 - humuor_loss: 1.2886 - offensive_loss: 1.1709 - motivational_loss: 0.6484 - overall_loss: 1.2824 - sarcasm_accuracy: 0.4944 - humuor_accuracy: 0.3581 - offensive_accuracy: 0.3926 - motivational_accuracy: 0.6486 - overall_accuracy: 0.446

70/70 [==============================] - 71s 1s/step - loss: 5.5736 - sarcasm_loss: 1.1835 - humuor_loss: 1.2886 - offensive_loss: 1.1709 - motivational_loss: 0.6484 - overall_loss: 1.2824 - sarcasm_accuracy: 0.4944 - humuor_accuracy: 0.3581 - offensive_accuracy: 0.3926 - motivational_accuracy: 0.6486 - overall_accuracy: 0.4465 - val_loss: 5.5387 - val_sarcasm_loss: 1.1534 - val_humuor_loss: 1.2961 - val_offensive_loss: 1.1770 - val_motivational_loss: 0.6585 - val_overall_loss: 1.2538 - val_sarcasm_accuracy: 0.5157 - val_humuor_accuracy: 0.3509 - val_offensive_accuracy: 0.3724 - val_motivational_accuracy: 0.6321 - val_overall_accuracy: 0.4628 - lr: 0.0010
Epoch 20/50
70/70 [==============================] - ETA: 0s - loss: 5.5735 - sarcasm_loss: 1.1835 - humuor_loss: 1.2886 - offensive_loss: 1.1708 - motivational_loss: 0.6484 - overall_loss: 1.2823 - sarcasm_accuracy: 0.4944 - humuor_accuracy: 0.3581 - offensive_accuracy: 0.3926 - motivational_accuracy: 0.6486 - overall_accuracy: 0.446

70/70 [==============================] - 72s 1s/step - loss: 5.5735 - sarcasm_loss: 1.1835 - humuor_loss: 1.2886 - offensive_loss: 1.1708 - motivational_loss: 0.6484 - overall_loss: 1.2823 - sarcasm_accuracy: 0.4944 - humuor_accuracy: 0.3581 - offensive_accuracy: 0.3926 - motivational_accuracy: 0.6486 - overall_accuracy: 0.4465 - val_loss: 5.5383 - val_sarcasm_loss: 1.1533 - val_humuor_loss: 1.2961 - val_offensive_loss: 1.1768 - val_motivational_loss: 0.6585 - val_overall_loss: 1.2537 - val_sarcasm_accuracy: 0.5157 - val_humuor_accuracy: 0.3509 - val_offensive_accuracy: 0.3724 - val_motivational_accuracy: 0.6321 - val_overall_accuracy: 0.4628 - lr: 0.0010
Epoch 21/50
70/70 [==============================] - ETA: 0s - loss: 5.5734 - sarcasm_loss: 1.1834 - humuor_loss: 1.2886 - offensive_loss: 1.1708 - motivational_loss: 0.6484 - overall_loss: 1.2822 - sarcasm_accuracy: 0.4944 - humuor_accuracy: 0.3581 - offensive_accuracy: 0.3926 - motivational_accuracy: 0.6486 - overall_accuracy: 0.446

70/70 [==============================] - 71s 1s/step - loss: 5.5734 - sarcasm_loss: 1.1834 - humuor_loss: 1.2886 - offensive_loss: 1.1708 - motivational_loss: 0.6484 - overall_loss: 1.2822 - sarcasm_accuracy: 0.4944 - humuor_accuracy: 0.3581 - offensive_accuracy: 0.3926 - motivational_accuracy: 0.6486 - overall_accuracy: 0.4465 - val_loss: 5.5383 - val_sarcasm_loss: 1.1533 - val_humuor_loss: 1.2961 - val_offensive_loss: 1.1768 - val_motivational_loss: 0.6585 - val_overall_loss: 1.2537 - val_sarcasm_accuracy: 0.5157 - val_humuor_accuracy: 0.3509 - val_offensive_accuracy: 0.3724 - val_motivational_accuracy: 0.6321 - val_overall_accuracy: 0.4628 - lr: 0.0010
Epoch 22/50
70/70 [==============================] - 42s 598ms/step - loss: 5.5733 - sarcasm_loss: 1.1834 - humuor_loss: 1.2885 - offensive_loss: 1.1708 - motivational_loss: 0.6484 - overall_loss: 1.2822 - sarcasm_accuracy: 0.4944 - humuor_accuracy: 0.3581 - offensive_accuracy: 0.3926 - motivational_accuracy: 0.6486 - overall_accuracy

70/70 [==============================] - 71s 1s/step - loss: 5.5734 - sarcasm_loss: 1.1834 - humuor_loss: 1.2886 - offensive_loss: 1.1708 - motivational_loss: 0.6484 - overall_loss: 1.2822 - sarcasm_accuracy: 0.4944 - humuor_accuracy: 0.3581 - offensive_accuracy: 0.3926 - motivational_accuracy: 0.6486 - overall_accuracy: 0.4465 - val_loss: 5.5379 - val_sarcasm_loss: 1.1532 - val_humuor_loss: 1.2961 - val_offensive_loss: 1.1767 - val_motivational_loss: 0.6585 - val_overall_loss: 1.2535 - val_sarcasm_accuracy: 0.5157 - val_humuor_accuracy: 0.3509 - val_offensive_accuracy: 0.3724 - val_motivational_accuracy: 0.6321 - val_overall_accuracy: 0.4628 - lr: 0.0010
Epoch 24/50
70/70 [==============================] - 46s 656ms/step - loss: 5.5733 - sarcasm_loss: 1.1834 - humuor_loss: 1.2885 - offensive_loss: 1.1708 - motivational_loss: 0.6484 - overall_loss: 1.2822 - sarcasm_accuracy: 0.4944 - humuor_accuracy: 0.3581 - offensive_accuracy: 0.3926 - motivational_accuracy: 0.6486 - overall_accuracy

In [91]:
!rm -rf ./model-0*

In [45]:
evaluate = model.evaluate(x = {"image_input": X_test, "text": X_text_test},
                    y = {"sarcasm": y_test.sarcasm, 
                         "humuor": y_test.humour, 
                         "offensive": y_test.offensive, 
                         "motivational": y_test.motivational, 
                         "overall": y_test.overall_sentiment},
                    batch_size=32,
                    verbose=2
                   )

44/44 - 6s - loss: 5.5593 - sarcasm_loss: 1.1695 - humuor_loss: 1.3104 - offensive_loss: 1.1731 - motivational_loss: 0.6440 - overall_loss: 1.2623 - sarcasm_accuracy: 0.5140 - humuor_accuracy: 0.3278 - offensive_accuracy: 0.3844 - motivational_accuracy: 0.6557 - overall_accuracy: 0.4359 - 6s/epoch - 142ms/step
